In [64]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("myApp").enableHiveSupport().config("hive", "true").getOrCreate()

23/04/10 11:10:05 WARN Utils: Your hostname, TIGER03531 resolves to a loopback address: 127.0.1.1; using 172.29.71.130 instead (on interface eth0)
23/04/10 11:10:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 11:10:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark.read.json("2015-summary.json")\
  .createOrReplaceTempView("flights") # DF => SQL

spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM flights GROUP BY DEST_COUNTRY_NAME
""")\
  .where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
  .count() # SQL => DF

12

In [4]:
spark.sql("SELECT 1 + 1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



In [10]:
spark.sql("select * from flights").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

### Creating Table using a Query

In [16]:
spark.sql("CREATE TABLE flights_from_select USING parquet AS SELECT * FROM flights").show()

++
||
++
++



In [17]:
spark.sql('select * from flights_from_select').show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [18]:
spark.sql("""CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5
""")

DataFrame[]

In [19]:
spark.sql("select * from partitioned_flights").show()

+-------------------+-----+-----------------+
|ORIGIN_COUNTRY_NAME|count|DEST_COUNTRY_NAME|
+-------------------+-----+-----------------+
|      United States|   15|            Egypt|
|            Romania|   15|    United States|
|            Croatia|    1|    United States|
|            Ireland|  344|    United States|
|              India|   62|    United States|
+-------------------+-----+-----------------+



### Creating External Tables

In [23]:
spark.sql("""CREATE EXTERNAL TABLE hive_flights_2
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION './data/flight-data-hive/' AS SELECT * FROM flights
""")

DataFrame[]

In [24]:
spark.sql('select * from hive_flights_2').show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

### Inserting into Tables

In [25]:
spark.sql("""
INSERT INTO flights_from_select
SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 20
""").show()

++
||
++
++



### Describing Table Metadata

In [28]:
spark.sql("DESCRIBE TABLE flights_csv").show()

+-------------------+---------+--------------------+
|           col_name|data_type|             comment|
+-------------------+---------+--------------------+
|  DEST_COUNTRY_NAME|   string|                null|
|ORIGIN_COUNTRY_NAME|   string|remember, the US ...|
|              count|   bigint|                null|
+-------------------+---------+--------------------+



In [31]:
spark.sql("SHOW PARTITIONS partitioned_flights").show()

+--------------------+
|           partition|
+--------------------+
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
+--------------------+



### Refreshing Table Metadata

In [33]:
spark.sql("""
REFRESH table partitioned_flights
""").show()

++
||
++
++



In [34]:
spark.sql("""
MSCK REPAIR TABLE partitioned_flights
""").show()

++
||
++
++



### Dropping Tables

In [36]:
spark.sql("""
DROP TABLE IF EXISTS flights_csv
""").show()

++
||
++
++



### Caching Tables

In [38]:
spark.sql("""
CACHE TABLE flights
""").show()

23/04/10 12:19:41 WARN CacheManager: Asked to cache already cached data.
++
||
++
++



In [39]:
spark.sql("""
UNCACHE TABLE FLIGHTS
""").show()

++
||
++
++



### Creating Views

In [46]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW just_usa_view AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
""").show()

++
||
++
++



In [43]:
spark.sql("""
CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
SELECT * FROM flights WHERE dest_country_name = 'United States'
""").show()

++
||
++
++



In [45]:
spark.sql("""
SHOW TABLES
""").show()

+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
|  default|flights_from_select|      false|
|  default|     hive_flights_2|      false|
|  default|partitioned_flights|      false|
|  default|               temp|      false|
|  default|              temp2|      false|
|         |            flights|       true|
|         |      just_usa_view|       true|
|         |      some_sql_view|       true|
+---------+-------------------+-----------+



In [50]:
spark.sql("""
SELECT * FROM just_usa_view
""").show()

+-----------------+--------------------+-----+
|DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+-----------------+--------------------+-----+
|    United States|             Romania|   15|
|    United States|             Croatia|    1|
|    United States|             Ireland|  344|
|    United States|               India|   62|
|    United States|           Singapore|    1|
|    United States|             Grenada|   62|
|    United States|        Sint Maarten|  325|
|    United States|    Marshall Islands|   39|
|    United States|            Paraguay|    6|
|    United States|           Gibraltar|    1|
|    United States|Federated States ...|   69|
|    United States|              Russia|  161|
|    United States|         Netherlands|  660|
|    United States|             Senegal|   42|
|    United States|              Angola|   13|
|    United States|            Anguilla|   38|
|    United States|             Ecuador|  300|
|    United States|              Cyprus|    1|
|    United S

In [51]:
spark.sql("""
EXPLAIN SELECT * FROM flights WHERE dest_country_name = 'United States'
""").show()

+--------------------+
|                plan|
+--------------------+
|== Physical Plan ...|
+--------------------+



In [53]:
spark.sql("""
EXPLAIN SELECT * FROM just_usa_view
""").show()

+--------------------+
|                plan|
+--------------------+
|== Physical Plan ...|
+--------------------+



### Dropping Views

In [54]:
spark.sql("DROP VIEW IF EXISTS just_usa_view").show()

++
||
++
++



### Creating Databases

In [57]:
spark.sql("""
SHOW DATABASES
""").show()

+---------+
|namespace|
+---------+
|  default|
|  some_db|
+---------+



In [56]:
spark.sql("""
CREATE DATABASE some_db
""").show()

23/04/10 12:35:39 WARN ObjectStore: Failed to get database some_db, returning NoSuchObjectException
23/04/10 12:35:39 WARN ObjectStore: Failed to get database some_db, returning NoSuchObjectException
23/04/10 12:35:39 WARN ObjectStore: Failed to get database some_db, returning NoSuchObjectException
++
||
++
++



In [61]:
spark.sql("""
USE some_db
""").show()

++
||
++
++



In [62]:
spark.sql("""
SHOW tables
""").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|         |      flights|       true|
|         |some_sql_view|       true|
+---------+-------------+-----------+



In [63]:
spark.sql("""
SELECT * FROM flights -- fails with table/view not found
""").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [65]:
spark.sql("""
select * from flights_from_select
""").show()

AnalysisException: Table or view not found: flights_from_select; line 2 pos 14;
'Project [*]
+- 'UnresolvedRelation [flights_from_select], [], false


In [66]:
spark.sql("""
SELECT * FROM default.flights_from_select
""").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [68]:
spark.sql("""
SELECT current_database()
""").show()

+------------------+
|current_database()|
+------------------+
|           some_db|
+------------------+



In [69]:
spark.sql("""
USE default;
""").show()

++
||
++
++



### Dropping Database

In [70]:
spark.sql("""
DROP DATABASE IF EXISTS some_db
""").show()

23/04/10 13:03:31 WARN TxnHandler: Cannot perform cleanup since metastore table does not exist
++
||
++
++



In [73]:
spark.sql("""
select * from flights
""").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

### Functions

In [75]:
spark.sql("""
SHOW FUNCTIONS
""").show()

+--------+
|function|
+--------+
|       !|
|      !=|
|       %|
|       &|
|       *|
|       +|
|       -|
|       /|
|       <|
|      <=|
|     <=>|
|      <>|
|       =|
|      ==|
|       >|
|      >=|
|       ^|
|     abs|
|    acos|
|   acosh|
+--------+
only showing top 20 rows



In [77]:
spark.sql("""
SHOW SYSTEM FUNCTIONS

""").show()

+--------+
|function|
+--------+
|       !|
|      !=|
|       %|
|       &|
|       *|
|       +|
|       -|
|       /|
|       <|
|      <=|
|     <=>|
|      <>|
|       =|
|      ==|
|       >|
|      >=|
|       ^|
|     abs|
|    acos|
|   acosh|
+--------+
only showing top 20 rows



In [79]:
spark.sql("""
SHOW USER FUNCTIONS
""").show()

+--------+
|function|
+--------+
+--------+



In [80]:
spark.sql("""
SHOW FUNCTIONS "s*"
""").show()

+------------------+
|          function|
+------------------+
|     schema_of_csv|
|    schema_of_json|
|               sec|
|            second|
|         sentences|
|          sequence|
|    session_window|
|               sha|
|              sha1|
|              sha2|
|         shiftleft|
|        shiftright|
|shiftrightunsigned|
|           shuffle|
|              sign|
|            signum|
|               sin|
|              sinh|
|              size|
|          skewness|
+------------------+
only showing top 20 rows



In [81]:
spark.sql("""
SHOW FUNCTIONS LIKE "collect*";
""").show()

+------------+
|    function|
+------------+
|collect_list|
| collect_set|
+------------+



In [83]:
spark.sql("""
SHOW FUNCTIONS LIKE "collect*";
""").show()

+------------+
|    function|
+------------+
|collect_list|
| collect_set|
+------------+



In [86]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# Define the UDF
def is_adult(age):
    if age >= 18:
        return True
    else:
        return False

# Register the UDF with the BooleanType return type
spark.udf.register("isAdult", is_adult, BooleanType())

<function __main__.is_adult(age)>

### Subqueries

In [87]:
spark.sql("""
SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5
""").show()

+-----------------+
|dest_country_name|
+-----------------+
|    United States|
|           Canada|
|           Mexico|
|   United Kingdom|
|            Japan|
+-----------------+



In [88]:
spark.sql("""
SELECT * FROM flights
WHERE origin_country_name IN (SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)
""").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Egypt|      United States|   15|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|             Algeria|      United States|    4|
|Turks and Caicos ...|      United States|  230|
|Saint Vincent and...|      United States|    1|
|               Italy|      United States|  382|
|            Pakistan|      United States|   12|
|             Iceland|      United States|  181|
|    Marshall Islands|      United States|   42|
|          Luxembourg|      United States|  155|
|            Honduras|      United States|  362|
|         The Bahama

In [89]:
spark.sql("""
SELECT * FROM flights f1
WHERE EXISTS (SELECT 1 FROM flights f2
WHERE f1.dest_country_name = f2.origin_country_name)
AND EXISTS (SELECT 1 FROM flights f2
WHERE f2.dest_country_name = f1.origin_country_name)
""").show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Croatia|    1|
|       United States|             Ireland|  344|
|               Egypt|       United States|   15|
|       United States|               India|   62|
|       United States|           Singapore|    1|
|       United States|             Grenada|   62|
|          Costa Rica|       United States|  588|
|             Senegal|       United States|   40|
|       United States|        Sint Maarten|  325|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|               Malta|       United States|    1|
|            Anguilla|       United States|   41|
|             Bolivia|       United States|   30|
|       United States|            Paraguay|    6|
|Turks and Caicos ...|       United States|  230|


In [90]:
spark.sql("""
SELECT *, (SELECT max(count) FROM flights) AS maximum FROM flights
""").show()

+--------------------+-------------------+-----+-------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|maximum|
+--------------------+-------------------+-----+-------+
|       United States|            Romania|   15| 370002|
|       United States|            Croatia|    1| 370002|
|       United States|            Ireland|  344| 370002|
|               Egypt|      United States|   15| 370002|
|       United States|              India|   62| 370002|
|       United States|          Singapore|    1| 370002|
|       United States|            Grenada|   62| 370002|
|          Costa Rica|      United States|  588| 370002|
|             Senegal|      United States|   40| 370002|
|             Moldova|      United States|    1| 370002|
|       United States|       Sint Maarten|  325| 370002|
|       United States|   Marshall Islands|   39| 370002|
|              Guyana|      United States|   64| 370002|
|               Malta|      United States|    1| 370002|
|            Anguilla|      Uni

In [ ]:
spark.sql("""

""").show()

In [ ]:
spark.sql("""

""").show()

In [ ]:
spark.sql("""

""").show()

In [ ]:
spark.sql("""

""").show()